# Financial Ratios of Stocks for Fundamental Analysis
#### Why am I doing this project:
As I invest in stocks and heard from my friend that Fundamental Analysis makes you more informed value-investor and I know python so why don't try to make my own basic screener.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [163]:
# For data manipulation
import pandas as pd

# To extract fundamental data
from bs4 import BeautifulSoup as bs
import re
import requests

# Creating the DataFrame
+ Add the list of Stocks and Ratios you are Intrested In

In [164]:
metric = ['P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

tickers = ['AMZN','GOOG','PG','KO','IBM','DG','XOM','KO','PEP','MT','NL','LPL']

df = pd.DataFrame(index=tickers,columns=metric)
df.head()

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IBM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Function to get Data from FinViz

In [165]:
for ticker in df.index:
    try:
        url = ("http://finviz.com/quote.ashx?t=" + ticker)
        req = Request(url = url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}) 
        response = urlopen(req)
        # Reading the contents of response into 'html'
        html = BeautifulSoup(response)
        # Getting specified fundamentals for tickers 
        for i in df.columns:
            # Find 'P/E' or other fundamentals in the soup and add it
            df.loc[ticker,i] = html.find(text = i).find_next(class_ = 'snapshot-td2').text
    except Exception as e:
        print(ticker,'not found')

In [166]:
df.head(1)

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,11.06,46.42,41.55,1.33,0.48,64.71,-,28.00%,9.50%,96.40%,9.80%


In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, AMZN to LPL
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   P/B          12 non-null     object
 1   P/E          12 non-null     object
 2   Forward P/E  12 non-null     object
 3   PEG          12 non-null     object
 4   Debt/Eq      12 non-null     object
 5   EPS (ttm)    12 non-null     object
 6   Dividend %   12 non-null     object
 7   ROE          12 non-null     object
 8   ROI          12 non-null     object
 9   EPS Q/Q      12 non-null     object
 10  Insider Own  12 non-null     object
dtypes: object(11)
memory usage: 1.4+ KB


#### Further work needs to be done as the value are recognized as STRINGS. This is beacuse the missing values are labelled as ' - ' and  ' % '. So we need to convert everything into numeric data.

In [168]:
df['Dividend %'] = df['Dividend %'].str.replace('%','')
df['ROE'] = df['ROE'].str.replace('%','')
df['ROI'] = df['ROI'].str.replace('%','')
df['EPS Q/Q'] = df['EPS Q/Q'].str.replace('%','')
df['Insider Own'] = df['Insider Own'].str.replace('%','')
# Stackoverflow helped me here
# Convert String to int to perform further calculations
df = df.apply(pd.to_numeric, errors='coerce')
print(df)
df.info()

        P/B    P/E  Forward P/E   PEG  Debt/Eq  EPS (ttm)  Dividend %   ROE  \
AMZN  11.06  46.42        41.55  1.33     0.48      64.71         NaN  28.0   
GOOG   7.03  24.93        23.05  1.19      NaN     103.81         NaN   NaN   
PG     8.71  27.91        24.62  3.93     0.81       5.66        2.20  31.6   
KO    12.12  27.62        23.57  3.82     1.88       2.25        2.70  38.4   
IBM    5.00  23.83        11.72  1.44     2.45       5.20        5.29   NaN   
DG     7.21  18.79        17.15  2.84     0.00      10.21        0.88  38.7   
XOM    2.04  14.19        12.44  2.44     0.35       5.39        4.60  -8.4   
KO    12.12  27.62        23.57  3.82     1.88       2.25        2.70  38.4   
PEP   14.54  29.94        23.10  3.86     2.51       5.62        2.55  49.8   
MT     0.63   2.24         4.80   NaN     0.17      13.46        0.99  33.2   
NL     0.86   7.28         6.97   NaN     0.00       0.83        3.96  12.3   
LPL    0.52   4.35         7.49  0.13     1.04      

### Data Gathering is done. Now Filtering good Companies

#### 1. Companies which are at low valuations

P/E < 15 and P/B < 1

In [169]:
df_filtered = df[(df['P/E']<15) & (df['P/B']<1)]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.63,2.24,4.80,NaN,0.17,13.46,0.99,33.2,25.2,289.3,44.8
NL,0.86,7.28,6.97,NaN,0.00,0.83,3.96,12.3,1.6,166.3,83.0
LPL,0.52,4.35,7.49,0.13,1.04,1.79,NaN,11.4,1.2,-68.3,51.1


#### 2. Further filter companies which have higher earning power
EPS Q/Q > 10%

In [170]:
df_filtered = df_filtered[df_filtered['EPS Q/Q'] > 10]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.63,2.24,4.80,NaN,0.17,13.46,0.99,33.2,25.2,289.3,44.8
NL,0.86,7.28,6.97,NaN,0.00,0.83,3.96,12.3,1.6,166.3,83.0


### Managment having substantial ownership in business
A high stake in the shares of the Managment is a positive sign as it shows the confidence of the promoters towards their own company.

Insider Own > 30

In [171]:
df = df[df['Insider Own'] > 30]
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.63,2.24,4.80,NaN,0.17,13.46,0.99,33.2,25.2,289.3,44.8
NL,0.86,7.28,6.97,NaN,0.00,0.83,3.96,12.3,1.6,166.3,83.0
LPL,0.52,4.35,7.49,0.13,1.04,1.79,NaN,11.4,1.2,-68.3,51.1


New thing i learned WEBSCRAPPING